In [5]:
# Import libraries & DEFINE FUNCTIONS 


import numpy as np
import pandas as pd
import gc
import math
import re
import datetime
from datetime import datetime
import sys
import random

#Web scraping imports 

import requests
from bs4 import BeautifulSoup as bs
import json
import urllib.request
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException as TOE
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.proxy import Proxy,ProxyType
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common import exceptions
from selenium.common.exceptions import WebDriverException as WDE



# For Viz

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Other 

from itertools import product
import collections
from collections import Counter
from itertools import permutations
from urllib.error import HTTPError, URLError
    
    
    
# --------- GET_PROXIES() & CHECK_PROXIES() were adapted from towardsdatascience.com 
    
    
def get_proxies():

    proxy_web_site = 'not_included'
    response = requests.get(proxy_web_site)
    page_html = response.text
    page_soup = bs(page_html, "html.parser")
    container = page_soup.find_all("div", {"class": "table-responsive"})[0]
    multi_by = [8*i for i in range(299)]
    proxies = set()
    
    for x in multi_by:
        country = container.find_all("td")[x+2].text 
        ip = container.find_all("td")[x].text
        port = container.find_all("td")[x+1].text
        if country == 'US':
            proxy = ip + ':' + port
            proxies.add(proxy)
        else:
            pass
                
    return proxies 

def check_proxies():
    
    working_proxies = set()
    proxies = get_proxies()            
    test_url = 'not_included'    
    for i in proxies:
        try:
            response = requests.get(test_url, proxies={"https": i}, timeout = 10)
            working_proxies.add(i)
        except:
            pass
    
    return working_proxies




# ------




def bettingSite1():
   

    # Get proxies, print proxies, test proxies
    prox_listIP = check_proxies()
    prox_listIP = list(prox_listIP)
    prox_listIP.insert(0, 'most_reliable_proxy')
    print(prox_listIP)
    
    
    # Variable for either starting or stopping the program
    red_green = 0
    while red_green == 0:     
        
        # if there are no more ips then it quits the program
        # Because we are popping [0] below if it doesnt work it automatically cycles the next ip into the [0] spot
        try:
            the_ip = prox_listIP[0]
            pass
        except:
            print("no proxies passed")
            quit()
            browser.quit()

        # Open Chrome in Stealth
        capabilities = webdriver.DesiredCapabilities.CHROME
        prox = Proxy()
        prox.proxy_type = ProxyType.MANUAL
        prox.autodetect = False
        prox.http_proxy = the_ip
        prox.ssl_proxy = the_ip
        prox.ftp_proxy = the_ip
        prox.add_to_capabilities(capabilities)

        option = webdriver.ChromeOptions()
        #For ChromeDriver version 79.0.3945.16 or over to go stealth
        # IF FOR SOME REASON THIS IS HERE TWICE option.add_argument('--disable-blink-features=AutomationControlled')
        #For working from a proxie
        option.add_argument('--proxy-server=%s' % the_ip)
        # THE THREE BELOW OR FOR AVOIDING CLOUDFLARE DETECTION
        option.add_experimental_option("excludeSwitches", ["enable-automation"])
        option.add_experimental_option('useAutomationExtension', False)
        option.add_argument("--disable-blink-features=AutomationControlled")


        path_to_driver = '/Users/blank/Desktop/chromedriver'
        browser = webdriver.Chrome(executable_path = path_to_driver, options=option, desired_capabilities= capabilities)

        # STEALTH 

        # Remove navigator.webdriver Flag using JavaScript
        browser.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")


        # browser.get("https://httpbin.org/ip")
        url = "sports_betting_site1"
        browser.get(url)

        # Checking to see if it actually loaded and if should continue with this IP
        try:
            WebDriverWait(browser,60).until(EC.presence_of_element_located((By.XPATH, '/html/body/*')))
            print("passed xpath test")
        except WDE:
            pass
        except:
            browser.quit()
            print("nothing loaded after 45 seconds")
            quit()

        try:
            browser.find_element_by_id('t')
            time.sleep(5)
            if browser.find_element_by_id('t'):
                prox_listIP.pop(0)
                print('popped from list at found t')
                browser.quit()
            else:
                pass
        except:
            pass

        try:
            browser.find_element_by_id('cf-wrapper')
            prox_listIP.pop(0)
            print('popped from list at found cf-wrapper')
            browser.quit()
        except:
            red_green += 1
                                        

    #Just wait a little
    time.sleep(5)

    # Scroll to bottom of the page
    browser.execute_script("window.scrollBy(0,document.body.scrollHeight)")

    # Make sure page loads if not try refresh three time unless there is a clear bad-proxy error
    main_page_load_test = 0
    refresh_count = 0

    while main_page_load_test != 1:
        # Wait for it to load the 
        try:
            main_page_load_test = 0
            WebDriverWait(browser,25).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[class*="add-to-betslip"]')))
            WebDriverWait(browser,5).until(EC.element_to_be_clickable((By.LINK_TEXT, 'Promotions')))
            main_page_load_test += 1
        except:
            browser.refresh()
            refresh_count += 1
            if refresh_count > 3:
                browser.quit()

    #change odds format 
    odds_drop_down = browser.find_element_by_css_selector('span[id="react-select-3--value-item"]')
    odds_drop_down.click()
    time.sleep(2)
    decimal_link = browser.find_element_by_css_selector('div[id="react-select-3--option-0"]')
    decimal_link.click()


    # CREATING LINKS AND LIST


    # Creating master list for BetFred
    sportsBettingSite1_masterList = []
    # For counting sports
    
    
    # --- LOOP STARTS HERE 
    
    
    
    # START LOOPING HERE 
    for sport in range(8):
    
       
        # Make sure things did not go stale and that connection is still active 
        browser.refresh()
        time.sleep(5)
            
        # Making sure its all a go and proceeding
        browser.refresh()
        try:
            WebDriverWait(browser,45).until(EC.element_to_be_clickable((By.LINK_TEXT, 'Promotions')))
            time.sleep(10)
        except:
            print("Promotions")
            browser.quit()
            quit()

        time.sleep(2)
        # Scroll to bottom of the page 
        browser.execute_script("window.scrollBy(0,document.body.scrollHeight)")
        time.sleep(2)

        # SCROLL THE PAGE TO THE TENNIS LINK
        browser.execute_script("arguments[0].scrollIntoView();", browser.find_element_by_link_text('Tennis'))
        # Wait for it to load the 
        try:
            WebDriverWait(browser,45).until(EC.element_to_be_clickable((By.LINK_TEXT, 'Tennis')))
        except:
            browser.quit()
        # SCROLL THE PAGE TO THE BASEBALL LINK - this makes it all visible
        browser.execute_script("arguments[0].scrollIntoView();", browser.find_element_by_link_text('Baseball'))
        time.sleep(2)

        # Hockey Link 
        hockey_link = browser.find_element_by_link_text('Ice Hockey')
        # Soccer Link 
        soccer_link = browser.find_element_by_link_text("Soccer")
        # UFC/MMA Link 
        ufc_mma_link = browser.find_element_by_link_text("Mixed Martial Arts")
        # Tennis Link 
        tennis_link = browser.find_element_by_link_text("Tennis")
        # Boxing Link 
        boxing_link = browser.find_element_by_link_text("Boxing")
        # Baseball Link
        baseball_link = browser.find_element_by_link_text("Baseball")
        # Basketball Link
        basketball_link = browser.find_element_by_link_text("Basketball")
        # Table Tennis Link 
        table_tennis_link = browser.find_element_by_link_text("Table Tennis")

        # Create sports_link_list for format1 sports
        sports_links = [hockey_link,ufc_mma_link,tennis_link,boxing_link,baseball_link,table_tennis_link,basketball_link,soccer_link]


        # Clicking on the link for each new sport
        try:
            sports_links[sport].click()
        except exceptions.StaleElementReferenceException as stale:
            print(stale)
            pass

        try:
            WebDriverWait(browser,45).until(EC.element_to_be_clickable((By.LINK_TEXT, 'MLB')))
        except:
            browser.quit()


        time.sleep(3)
        browser.execute_script("window.scrollBy(0,document.body.scrollHeight)")
        time.sleep(3)


        # check if there are any 'load more' links 
        try:
            load_more = browser.find_elements_by_link_text("Load More...")
            for x in load_more:
                x.click()
                time.sleep(2) 
        except:
            print(f"did not find load more button for: {sports_links[sport]}")
            pass


        # Finding info
        all_events = browser.find_elements_by_css_selector('div[class="event-list__item-link"]')
        for x in all_events:
            try:
                location = browser.current_url
                event_html = x.get_attribute('outerHTML')
                event_soup = bs(event_html, 'html.parser')
                spans = event_soup.find_all('span')
                if sport == 6: # Basketball
                    side_1_name = spans[14]
                    side_2_name = spans[19]
                    side_1_odds = spans[16] 
                    side_2_odds = spans[21]
                elif sport == 7: # Soccer 
                    side_1_name = spans[2]
                    side_2_name = spans[12]
                    side_1_odds = spans[4] 
                    side_2_odds = spans[14]
                else: # The Rest 
                    side_1_odds = spans[4] 
                    side_2_odds = spans[9]
                    side_1_name = spans[2]
                    side_2_name = spans[7]
                sport_name = browser.find_elements_by_css_selector('a[role="menuitem"]')[1]
                event_name = side_1_name.text + ' vs ' + side_2_name.text
                event_date_time = x.find_element_by_css_selector('div[class$="event-market-footer"]')
                #create list and print for test 
                each_event_list = [url,location,sport_name.text,event_name,event_date_time.text,side_1_name.text,side_1_odds.text,side_2_name.text,side_2_odds.text]
                sportsBettingSite1_masterList.append(each_event_list)
                time.sleep(2)
            except:
                pass

            
    return sportsBettingSite1_masterList
    
    # Ends and Quits Browser
    print("completed bet_fred")
    browser.quit()

# --- RUNS HERE    

sportsBettingSite1_List = bettingSite1()